## Text Generation Inference using Online Endpoints - Dolly

This sample shows how to deploy `text-generation` type models to an online endpoint for inference.

### Task
`text-generation`  is the task of producing new text. These models can, for example, fill in incomplete text or paraphrase. Some common applications of text generation are code generation and story generation.

### Model
Models that can perform the `text-generation` task are tagged with `task: text-generation`. We will use the `mistralai-Mistral-7B-v01` model in this notebook. If you opened this notebook from a specific model card, remember to replace the specific model name. If you don't find a model that suits your scenario or domain, you can discover and [import models from HuggingFace hub](../../import/import_model_into_registry.ipynb) and then use them for inference. 

### Inference data
We will use the [book corpus](https://huggingface.co/datasets/bookcorpus) dataset.

### Outline
* Set up pre-requisites.
* Pick a model to deploy.
* Download and prepare data for inference. 
* Deploy the model for real time inference.
* Test the endpoint
* Clean up resources.

### 1. Set up pre-requisites
* Install dependencies
* Connect to AzureML Workspace. Learn more at [set up SDK authentication](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-setup-authentication?tabs=sdk). Replace  `<WORKSPACE_NAME>`, `<RESOURCE_GROUP>` and `<SUBSCRIPTION_ID>` below.
* Connect to `azureml` system registry

In [67]:
from azure.ai.ml import MLClient
from azure.identity import (
    DefaultAzureCredential,
    InteractiveBrowserCredential,
    ClientSecretCredential,
)
from azure.ai.ml.entities import AmlCompute
import time

try:
    credential = DefaultAzureCredential()
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    credential = InteractiveBrowserCredential()

workspace_ml_client = MLClient(
    credential,
    subscription_id="72c03bf3-4e69-41af-9532-dfcdc3eefef4",
    resource_group_name="shared-finetuning-rg",
    workspace_name="v-mathota",
)
# the models, fine tuning pipelines and environments are available in the AzureML system registry, "azureml"
registry_ml_client = MLClient(credential, registry_name="azureml")

### 2. Pick a model to deploy

Browse models in the Model Catalog in the AzureML Studio, filtering by the `text-generation` task. In this example, we use the `mistralai-Mistral-7B-v01` model. If you have opened this notebook for a different model, replace the model name and version accordingly. 

In [69]:
model_name = "mistralai-Mistral-7B-v01"
version_list = list(registry_ml_client.models.list(model_name))
if len(version_list) == 0:
    print("Model not found in registry")
else:
    model_version = version_list[0].version
    foundation_model = registry_ml_client.models.get(model_name, model_version)
    print(
        "\n\nUsing model name: {0}, version: {1}, id: {2} for inferencing".format(
            foundation_model.name, foundation_model.version, foundation_model.id
        )
    )



Using model name: mistralai-Mistral-7B-v01, version: 3, id: azureml://registries/azureml/models/mistralai-Mistral-7B-v01/versions/3 for inferencing


### 3. Download and prepare data for inference.

The next few cells show basic data preparation:
* Visualize some data rows
* Save few samples in the format that can be passed as input to the online-inference endpoint.

In [3]:
# Download a small sample of the dataset into the ./book-corpus-dataset directory
%run ./book-corpus-dataset/download-dataset.py --download_dir ./book-corpus-dataset

c:\Users\v-mathota\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading train split of bookcorpus dataset...


In [70]:
# load the ./book-corpus-dataset/train.jsonl file into a pandas dataframe and show the first 5 rows
import pandas as pd

pd.set_option(
    "display.max_colwidth", 0
)  # set the max column width to 0 to display the full text
train_df = pd.read_json("./book-corpus-dataset/train.jsonl", lines=True)
train_df.head(2)

,text
0,he had devoted hours to holding her and spoiling her rotten .
1,"he 's just a boy playing at being a man , '' she replied , glancing down at the table ."


### 4. Deploy the model to an online endpoint
Online endpoints give a durable REST API that can be used to integrate with applications that need to use the model.

In [71]:
import time, sys
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    ProbeSettings,
    OnlineRequestSettings,
)

# Create online endpoint - endpoint names need to be unique in a region, hence using timestamp to create unique endpoint name
timestamp = int(time.time())
online_endpoint_name = "text-generation-" + str(timestamp)
# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="Online endpoint for "
    + foundation_model.name
    + ", for text-generation task",
    auth_mode="key",
)
workspace_ml_client.begin_create_or_update(endpoint).wait()

In [72]:
# create a deployment
demo_deployment = ManagedOnlineDeployment(
    name="demo",
    endpoint_name=online_endpoint_name,
    model=foundation_model.id,
    instance_type="Standard_NC12s_v3",
    instance_count=1,
    liveness_probe=ProbeSettings(initial_delay=500, period=300,timeout=300, failure_threshold=10),
    request_settings=OnlineRequestSettings(
        request_timeout_ms=90000,max_queue_wait_ms = 90000, max_concurrent_requests_per_instance=1
    ),
)
workspace_ml_client.online_deployments.begin_create_or_update(demo_deployment).wait()
endpoint.traffic = {"demo": 100}
workspace_ml_client.begin_create_or_update(endpoint).result()

Check: endpoint text-generation-1701101634 exists


....................................................................................................................................................................

ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://text-generation-1701101634.australiaeast.inference.ml.azure.com/score', 'openapi_uri': 'https://text-generation-1701101634.australiaeast.inference.ml.azure.com/swagger.json', 'name': 'text-generation-1701101634', 'description': 'Online endpoint for mistralai-Mistral-7B-v01, for text-generation task', 'tags': {}, 'properties': {'azureml.onlineendpointid': '/subscriptions/72c03bf3-4e69-41af-9532-dfcdc3eefef4/resourcegroups/shared-finetuning-rg/providers/microsoft.machinelearningservices/workspaces/v-mathota/onlineendpoints/text-generation-1701101634', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/72c03bf3-4e69-41af-9532-dfcdc3eefef4/providers/Microsoft.MachineLearningServices/locations/australiaeast/mfeOperationsStatus/oe:0b59fe7d-5d45-46aa-af38-dcb6431d7b0b:fbc7fbad-c9d0-4385-a997-3ff3655ecf7c?api-version=2022-02-01-preview'}, 'print_as_yaml': Tru

### 5. Test the endpoint with sample data

We will fetch some sample data from the test dataset and submit to online endpoint for inference. We will then show the display the scored labels alongside the ground truth labels

In [73]:
import json
import os

# read the ./book-corpus-dataset/train.jsonl file into a pandas dataframe
df = pd.read_json("./book-corpus-dataset/train.jsonl", lines=True)
# escape single and double quotes in the text column
df["text"] = df["text"].str.replace("'", "\\'").str.replace('"', '\\"')
# pick 1 random row
sample_df = df.sample(1)
# create a json object with the key as "inputs" and value as a list of values from the article column of the sample_df dataframe
sample_json = {"inputs": sample_df["text"].tolist()}
# save the json object to a file named sample_score_dolly.json in the ./book-corpus-dataset folder
test_json = {"input_data": {"input_string": sample_df["text"].tolist()}}
print(test_json, "test_json")
# save the json object to a file named sample_score_dolly.json in the ./book-corpus-dataset folder
with open(
    os.path.join(".", "book-corpus-dataset", "sample_score_mistral.json"), "w"
) as f:
    json.dump(test_json, f)
sample_df.head()

{'input_data': {'input_string': ['`` we had great chemistry together .']}} test_json


,text
2,`` we had great chemistry together .


In [74]:
# score the sample_score_dolly.json file using the online endpoint with the azureml endpoint invoke method
response = workspace_ml_client.online_endpoints.invoke(
    endpoint_name=online_endpoint_name,
    deployment_name="demo",
    request_file="./book-corpus-dataset/sample_score_mistral.json",
)
print("raw response: \n", response, "\n")
# convert the json response to a pandas dataframe
response_df = pd.read_json(response)
response_df.head()

raw response: 
 [{"0": "`` we had great chemistry together .\n\nDoes anyone else like this player? I for one really enjoy his energy"}] 



C:\Users\v-mathota\AppData\Local\Temp\ipykernel_31748\2380975773.py:9: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  response_df = pd.read_json(response)


,0
0,`` we had great chemistry together .\n\nDoes anyone else like this player? I for one really enjoy his energy


### 6. Delete the online endpoint
Don't forget to delete the online endpoint, else you will leave the billing meter running for the compute used by the endpoint

In [ ]:
workspace_ml_client.online_endpoints.begin_delete(name=online_endpoint_name).wait()